# Documentation

The `ThresholdOptimizer` class optimizes thresholds for the `AngleAnalyzer` algorithm by systematically testing different combinations and scoring them based on specified criteria. It iteratively evaluates threshold combinations using a `DataAnalyzer` object, calculating scores derived from linear regression values and weighted metrics. The class then selects the best-scoring threshold combinations and exports the tested combinations to a CSV file. By providing insights into the trade-offs between different threshold settings, this class enables the fine-tuning of the `AngleAnalyzer` to achieve optimal performance.

Input:
- `.csv` file of format (processed using `FileConverter`) `timestamp [UTC],relative altitude [m],horizontal velocity [m/s],vertical velocity [m/s],distance to takeoff [km],longitude,latitude`

Output:
- `.csv` file of format `angle_past_threshold,angle_future_threshold,average_r_value,average_p_value,average_std_err,score,data_loss`

# Imports

In [ ]:
import os
import sys
import pandas as pd

# AI content (GitHub Copilot, 02/07/2024), verified and adapted by Nicolas Huber.
src_directory: str = os.path.join(os.getcwd(), "..")
sys.path.append(src_directory)

import constants as constants
import helpers.data_analyzer as dataanalyzer
import helpers.optimize_thresholds as optimizer
import helpers.data_visualizer as datavisualizer

# Variables

In [ ]:
CSV_FILE: str = "CSV_FILE"

# Conditions

In [ ]:
print("The automated threshold optimizer is executed for the following configuration:")
print(f"--> CSV file: {CSV_FILE}")
print(f"--> Runtime estimation: {constants.OPTIMIZATION_RUNTIME_ESTIMATION} seconds per iteration")
print(f"--> Optimization limit: {constants.OPTIMIZATION_LIMIT}")
print(f"--> Optimization step size: {constants.OPTIMIZATION_STEPS}")

# Initialisation

In [ ]:
Optimizer: optimizer.ThresholdOptimizer = optimizer.ThresholdOptimizer(
    csv_file=CSV_FILE,
    r_value_weight=constants.R_VALUE_WEIGHT,
    p_value_weight=constants.P_VALUE_WEIGHT,
    std_error_weight=constants.STD_ERROR_WEIGHT,
    limit=constants.OPTIMIZATION_LIMIT,
    steps=constants.OPTIMIZATION_STEPS,
    runtime_estimation=constants.OPTIMIZATION_RUNTIME_ESTIMATION
)
DataAnalyzer: dataanalyzer.DataAnalyzer = Optimizer.construct_data_analyzer()
Visualizer: datavisualizer.DataVisualizer = datavisualizer.DataVisualizer()

# Data Import

In [ ]:
data: pd.DataFrame = DataAnalyzer.read_csv_data()

# Visualize Flight

In [ ]:
Visualizer.visualize_points_colored(data)

# Optimization

In [ ]:
optimization = Optimizer.optimize_thresholds(data=data, DataAnalyzer=DataAnalyzer)
Optimizer.export_to_csv(results=optimization)
optimized_index: int = Optimizer.calculate_optimized_data_loss(data=optimization)

# Dataset

Ordered in descending order by score.

In [ ]:
optimization

# Visualisation

In [ ]:
Visualizer.visualize_optimization_linear_regression(data=optimization)
Visualizer.visualize_score_by_data_loss(data=optimization, index=optimized_index)
Visualizer.visualize_optimization_score(data=optimization)
Visualizer.visualize_optimization_rvalues(data=optimization)
Visualizer.visualize_optimization_pvalues(data=optimization)
Visualizer.visualize_optimization_stderrs(data=optimization)

# Report

In [ ]:
print("Individual thresholds with the best score:")
print(f"--> past_threshold_optimized: {Optimizer.past_threshold_optimized}")
print(f"--> future_threshold_optimized: {Optimizer.future_threshold_optimized}")
print()

print("Below is a tabular overview of the 5 best scores and their thresholds. This information is more meaningful here, as in the analysis later for the evaluation of a point, both the future and the past are taken into account, and thus the score considers the interaction of the two thresholds.")

In [ ]:
Optimizer.best_scores

In [ ]:
print(f"The best performing thresholds are {Optimizer.best_scores.iloc[0, 0]} (angle_past_threshold) and {Optimizer.best_scores.iloc[0, 1]} (angle_future_threshold) with a score of {Optimizer.best_scores.iloc[0, 5]}.")
print()
print(f"Another good performing set of thresholds can be found by comparing the data loss relative to the scores, which are directly related to the thresholds. In this case, the best performing thresholds are {optimization.iloc[optimized_index, 0]} (angle_past_threshold) and {optimization.iloc[optimized_index, 1]} (angle_future_threshold) with a score of {optimization.iloc[optimized_index, 5]} and a data loss of {optimization.iloc[optimized_index, 6]}. The bigger the difference between the score and the data loss, the better the thresholds are. This is the case because the precison of the thresholds is overall better if less data is lost, even if there is a small decrease in the score.")

# System Info

In [ ]:
print(f"@ Author {constants.AUTHOR}")
print(f"@ Author Email {constants.AUTHOR_EMAIL}")
print(f"@ Author URL {constants.AUTHOR_URL}")
print(f"@ GitHub URL {constants.GITHUB_URL}")